In [64]:
# AR
# 11.2019

In [65]:
# http://theo.cs.ovgu.de/lehre/lehre2019w/reasoning/05-uebungsblatt.pdf

In [66]:
from gavel.logic.logic import BinaryFormula, Constant, PredicateExpression, BinaryConnective, FunctorExpression, DefinedConstant

In [67]:
from gavel.prover.base.interface import BaseProverInterface

In [68]:
from gavel.logic.proof import Proof
from gavel.logic.proof import ProofStep

In [69]:
from gavel.dialects.tptp.parser import TPTPProblemParser

problem_parser = TPTPProblemParser()

In [70]:
# Convert string to gavel.logic.problem.Problem
def read_prob(st):
    x = problem_parser.parse(st)
    return next(x)

In [173]:
p1 = read_prob(open('../P4/problem1.p', 'r').readlines())
p2 = read_prob(open('../P4/problem2.p', 'r').readlines())
p3 = read_prob(open('../P4/problem3.p', 'r').readlines())
p4 = read_prob(open('../P4/problem4.p', 'r').readlines())

p12 = read_prob(open('problem1.p', 'r').readlines())
p22 = read_prob(open('problem2.p', 'r').readlines())
p32 = read_prob(open('problem3.p', 'r').readlines())

In [174]:
# Convert premises into list of strings
def formulas(p):
    if type(p)==list:
        return [str(i.formula) for i in p]
    return str(p.formula)

In [175]:
formulas(p32.premises)

['((p(e)) & ((p(b)) & (p(d)))) => ($false)',
 '(p(e)) => (p(d))',
 '($true) => (p(f))',
 '(p(a)) => ($false)',
 '(p(c)) => (p(e))',
 '($true) => (p(c))',
 '($true) => ((f) = (b))']

In [176]:
class Horn(BaseProverInterface):
    def _submit_problem(self, problem_instance, *args, **kwargs):
        # Call your prov:qer here
        sol = Solver()
        result = sol.prove(problem_instance)
        return result

In [202]:
ATOMS = [Constant, DefinedConstant, str]

In [203]:
def st(x):
    if x in [str, DefinedConstant]:
        return str(x)
    return str(x.symbol)

In [207]:
class Solver():
    def __init__(self):
        self.oldEq = {}
        self.equiv = {}
        self.func = {}
        self.oldS = set()
        self.S = {'$true'}
        self.oldPreds = {}
        self.preds = {}
        self.G = 0
        
        self.steps = [{'$true'}]
        
    def functor(self, f):
        if type(f) in ATOMS:
            return st(f)
        
        if all([type(i) in ATOMS for i in f.arguments]):
            if str(f) not in self.func:
                k = f'K{self.G}'
                self.func[str(f)] = k
                self.equiv[k] = set([k])
                self.G += 1
                return k
            return self.func[str(f)]

        l = [self.functor(i) for i in f.arguments]

        return self.functor(FunctorExpression(f.functor, l))
    
    def eq(self, x, y): 
        x = self.functor(x)
        y = self.functor(y)
        
        if x not in self.equiv[y]:
            self.equiv[x] = self.equiv[y]
            self.equiv[x].add(x)
        
    def check(self, exp):
        if type(exp) in [Constant, DefinedConstant]:
            return str(exp) in self.oldS
        
        if type(exp) == FunctorExpression:
            return self.functor(exp) in self.oldS
        
        elif type(exp) == PredicateExpression:
            p = str(exp.predicate)
            if p in self.oldPreds:
                v = tuple(self.functor(i) for i in exp.arguments)
                for x in self.oldPreds[p]:
                    assert len(v) == len(x), f'Non matching lengths. v: {v}, x: {x}, p:{p}'
                    if all(v[i] in self.oldEq[x[i]] for i in range(len(v))):
                        return True
            return False
        
        elif type(exp) == BinaryFormula:
            return self.check(exp.left) and self.check(exp.right) # Only supports and for the time being
        print('?')
        print(type(exp))
                 
    def prepare(self, exp):
        if type(exp) == Constant:
            k = str(exp.symbol)
            if k not in self.equiv:
                self.equiv[k] = set(str(exp))
            return
        if type(exp) == BinaryFormula:
            self.prepare(exp.left)
            self.prepare(exp.right)
            return
        if type(exp) in (PredicateExpression, FunctorExpression):
            for i in exp.arguments:
                self.prepare(i)
        
            
    def prove(self, problem, *args, **kwargs):
        lst = [i.formula for i in problem.premises]
        for exp in lst:
            self.prepare(exp)

        conj = problem.conjecture.formula
        
        neweq = False
        while self.S != self.oldS or neweq:
            neweq = False
            self.steps.append(set())
            self.oldS = self.S.copy()
            self.oldEq = self.equiv.copy()
            self.oldPreds = self.preds.copy()
            
            for exp in lst:
                r = exp.right
                
                # We already know
                if type(r) == BinaryFormula and r.operator == BinaryConnective.EQ:
                    x = self.functor(r.left)
                    y = self.functor(r.right)
                    if x in self.equiv[y]:
                        continue
                elif type(r) == DefinedConstant and str(r) in self.S:
                    continue
                elif type(r) == PredicateExpression:
                    p = str(r.predicate)
                    if p in self.preds:
                        v = tuple(self.functor(i) for i in r.arguments)
                        for x in self.preds[p]:
                            assert len(v) == len(x), f'Non matching lengths. v: {v}, x: {x}, p:{p}'
                            if all(v[i] in self.equiv[x[i]] for i in range(len(v))):
                                continue
                
                # We don't know
                if self.check(exp.left):
                    
                    if type(r) == BinaryFormula and r.operator == BinaryConnective.EQ:
                        self.steps[-1].update({str(r)})
                        self.eq(r.left, r.right)
                        neweq = True

                    elif type(r) == DefinedConstant:
                        self.S.add(str(r))

                    elif type(r) == PredicateExpression:
                        self.S.add(str(r))
                        p = str(r.predicate)
                        if p not in self.preds:
                            self.preds[p] = set()
                        v = tuple(self.functor(i) for i in r.arguments)
                        self.preds[p].add(v)
                    else:
                        print("???")
                        print(type(r))
                    if type(conj) == type(r) and conj == r:
                        self.steps[-1].update(self.S - self.oldS)
                        return Proof(steps=self.steps)
            self.steps[-1].update(self.S - self.oldS)

In [205]:
hop = Horn()

In [206]:
for prob in [p1, p2, p3, p4, p12, p22, p32]:
    print("Premises")
    print(formulas(prob.premises))
    print(f"\nConjecture: {str(prob.conjecture.formula)}")
    p = hop.prove(prob)
    if p:
        print("Proof found!")
        for i in p.steps:
            print(i)
    else:
        print("No proof found")
    print('\n')

Premises
['(p(a)) => (q(a))', '(q(a)) => ($false)', '($true) => (p(a))']

Conjecture: $false
Proof found!
{'$true'}
{'p(a)'}
{'q(a)'}
{'$false'}


Premises
['((p(a)) & ((p(b)) & (p(d)))) => ($false)', '(p(e)) => ($false)', '(p(c)) => (p(a))', '(p(c)) => ($false)', '(p(a)) => (p(d))']

Conjecture: $false
No proof found


Premises
['((p(e)) & ((p(b)) & (p(d)))) => ($false)', '(p(e)) => (p(d))', '($true) => (p(f))', '(p(a)) => ($false)', '(p(c)) => (p(e))', '($true) => (p(c))']

Conjecture: $false
No proof found


Premises
['((p(e)) & ((p(b)) & (p(d)))) => ($false)', '(p(e)) => (p(d))', '($true) => (p(b))', '(p(a)) => ($false)', '(p(c)) => (p(e))', '($true) => (p(c))']

Conjecture: $false
Proof found!
{'$true'}
{'p(b)', 'p(c)'}
{'p(e)'}
{'p(d)'}
{'$false'}


Premises
['($true) => ((a) = (b))', '($true) => (p(a))', '(p(b)) => ($false)']

Conjecture: $false
Proof found!
{'$true'}
{'p(a)', '(a) = (b)'}
{'$false'}


Premises
['($true) => ((a) = (d))', '($true) => (p(g(f(a), b)))', '($true) =>

In [ ]:
big = read_prob(open('../big.p').readlines())

In [ ]:
hop.prove(big)

In [61]:
## Alternative solver class with actual ProofStep Objects. This allows us to merge the old/new variables

class Solver():
    def __init__(self):
        self.equiv = {}
        self.func = {}
        self.S = {'$true'}
        self.preds = {}
        self.G = 0
        
        self.steps = [ProofStep(DefinedConstant.VERUM)]
        
    def functor(self, f):
        if type(f) in ATOMS:
            return st(f)
        
        if all([type(i) in ATOMS for i in f.arguments]):
            if str(f) not in self.func:
                k = f'K{self.G}'
                self.func[str(f)] = k
                self.equiv[k] = set([k])
                self.G += 1
                return k
            return self.func[str(f)]

        l = [self.functor(i) for i in f.arguments]

        return self.functor(FunctorExpression(f.functor, l))
    
    def eq(self, x, y): 
        x = self.functor(x)
        y = self.functor(y)
        
        #self.steps[-1].add(f"{x}={y}")
        
        if x not in self.equiv[y]:
            self.equiv[x] = self.equiv[y]
            self.equiv[x].add(x)
        
    def check(self, exp):
        if type(exp) in [Constant, DefinedConstant]:
            return str(exp) in self.S
        
        if type(exp) == FunctorExpression:
            return self.functor(exp) in self.S
        
        elif type(exp) == PredicateExpression:
            p = str(exp.predicate)
            if p in self.preds:
                v = tuple(self.functor(i) for i in exp.arguments)
                for x in self.preds[p]:
                    assert len(v) == len(x), f'Non matching lengths. v: {v}, x: {x}, p:{p}'
                    if all(v[i] in self.equiv[x[i]] for i in range(len(v))):
                        return True
            return False
        
        elif type(exp) == BinaryFormula:
            return self.check(exp.left) and self.check(exp.right) # Only supports and for the time being
        print('?')
        print(type(exp))
                 
    def prepare(self, exp):
        if type(exp) == Constant:
            k = str(exp.symbol)
            if k not in self.equiv:
                self.equiv[k] = set(str(exp))
            return
        if type(exp) == BinaryFormula:
            self.prepare(exp.left)
            self.prepare(exp.right)
            return
        if type(exp) in (PredicateExpression, FunctorExpression):
            for i in exp.arguments:
                self.prepare(i)
        
            
    def prove(self, problem, *args, **kwargs):
        lst = [i.formula for i in problem.premises]
        for exp in lst:
            self.prepare(exp)

        conj = problem.conjecture.formula
        
        update = True
        while update:
            update = False
            
            for exp in lst:
                r = exp.right
                
                # We already know
                if type(r) == BinaryFormula and r.operator == BinaryConnective.EQ:
                    x = self.functor(r.left)
                    y = self.functor(r.right)
                    if x in self.equiv[y]:
                        continue
                elif type(r) == DefinedConstant and str(r) in self.S:
                    continue
                elif type(r) == PredicateExpression:
                    p = str(r.predicate)
                    if p in self.preds:
                        v = tuple(self.functor(i) for i in r.arguments)
                        match = False
                        for x in self.preds[p]:
                            if all(v[i] in self.equiv[x[i]] for i in range(len(v))):
                                match = True
                                break
                        if match:
                            continue
                                
                
                # We don't know
                if self.check(exp.left):
                    update = True
                    
                    if type(r) == BinaryFormula and r.operator == BinaryConnective.EQ:
                        self.eq(r.left, r.right)

                    elif type(r) == DefinedConstant:
                        self.S.add(str(r))

                    elif type(r) == PredicateExpression:
                        self.S.add(str(r))
                        p = str(r.predicate)
                        if p not in self.preds:
                            self.preds[p] = set()
                        v = tuple(self.functor(i) for i in r.arguments)
                        self.preds[p].add(v)
                    else:
                        print("???")
                        print(type(r))
                    self.steps.append(ProofStep(exp))
                    if type(conj) == type(r) and conj == r:
                        return Proof(steps=self.steps)
        